# Statsmodels
* https://www.kaggle.com/ojwatson/mixed-models

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.tools.sm_exceptions import ConvergenceWarning

In [ ]:
# Formula indicates mean weight as a linear function of time. Random intercept for each pig. 
data = sm.datasets.get_rdataset('dietox', 'geepack').data
md = smf.mixedlm("Weight ~ Time", data, groups=data["Pig"], re_formula="~Time")
mdf = md.fit(method=["lbfgs"])
print(mdf.summary())

In [ ]:
rmse = mdf.fittedvalues

# Stats models on the FQI part

In [ ]:
with open('train_tuples.json', 'r') as f:
    train_dict = json.load(f)
with open('test_tuples.json', 'r') as f:
    test_dict = json.load(f)

In [ ]:
train_df = pd.DataFrame.from_dict(train_dict)
test_df = pd.DataFrame.from_dict(test_dict)
train_df.columns

In [ ]:
y = train_df['r']
X = train_df[['a0', 'a1', 's0', 's1', 's2', 's3', 's4', 's5', 's6', 's7', 's8', 's9']]
groups = train_df['ds']s

In [ ]:
model = sm.MixedLM(endog=y, exog=X, groups=groups)
result = model.fit()

In [ ]:
testX = test_df[['a0', 'a1', 's0', 's1', 's2', 's3', 's4', 's5', 's6', 's7', 's8', 's9']]
result.predict(exog=testX)